In [1]:
# A code to scrape name from 35mmc.com
# scrape for a defined level
# Aing KK.

from tqdm import tqdm
import concurrent.futures
import requests
from bs4 import BeautifulSoup
import mysql.connector
import time

In [2]:
def crawl_level(db, url, level):
    # depth limit for crawling url
    if level <= 7:
        webReq = requests.get(url)
        soup_obj = BeautifulSoup(webReq.text, "html.parser")
        if soup_obj.find('article').text != None:
            raw_text = soup_obj.find('article').text
            # get title
            get_title = soup_obj.find('h1', {'class':'entry-title'}).text
            print("Current level : ", level, " at : ", get_title)
            # store to db
            db.store_db(get_title, url, raw_text)
            # time.sleep(0.500)
            return [i.find('a')['href'] for i in soup_obj.find_all('div', {'class':'sp-col-4'})]
        else:
            pass
    else:
        return []

In [65]:
def crawl_3level(db, url, level):
    for link in crawl_level(db, url, level):
        crawl_3level(db, link, level+1)
    # print("level : ", level, " Completed")
        

In [4]:
class databasePipeline:
    def __init__(self, host, user, password, database, table_name):
        print("Initialize Database Pipeline")
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.table_name = table_name
        self.create_connection()
        self.curr = self.conn.cursor()
        self.create_table()

    def create_connection(self):
        """Create a connection to the database"""
        print("Creating Connection . . .")        self.conn = mysql.connector.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            database=self.database
        )
        print("Connceted.")
    
    def create_table(self):
        print("Creating table . . .")
        self.curr.execute("""DROP TABLE IF EXISTS 35mmc_raw""")
        self.curr.execute("""create table 35mmc_raw(
            title text,
            url text,
            raw_text text
        )""")
        print("Done Table.")

    def store_db(self, title, url, raw_text):
        self.curr.execute(
            """INSERT INTO {} (title, url, raw_text) VALUES (%s, %s, %s)""".format(self.table_name),
            (title, url, raw_text)
        )
        self.conn.commit()  

In [5]:
start_url = 'https://www.35mmc.com/20/05/2016/halina-af700-review/'

In [6]:
db = databasePipeline('localhost', 'root', 'aingg', '35mmc', '35mmc_raw')

Initialize Database Pipeline
Creating Connection . . .
Connceted.
Creating table . . .
Done Table.


In [ ]:
start_level = 1 
crawl_3level(db, start_url, start_level)
print("DONE")

In [13]:
from bs4 import BeautifulSoup
import requests

# url = 'https://www.35mmc.com/20/05/2016/halina-af700-review/'
# url = "https://fstoppers.com/film/how-was-i-missing-medium-format-point-and-shoot-short-term-review-fujifilm-ga645-470116"
# url = "https://www.trustedreviews.com/reviews/nikon-z9"
# url = "https://artyt.me/2022/03/21/%e0%b8%a3%e0%b8%b5%e0%b8%a7%e0%b8%b4%e0%b8%a7-10-%e0%b8%81%e0%b8%a5%e0%b9%89%e0%b8%ad%e0%b8%87%e0%b8%9f%e0%b8%b4%e0%b8%a5%e0%b9%8c%e0%b8%a1%e0%b8%97%e0%b8%b5%e0%b9%88%e0%b8%84%e0%b8%a7%e0%b8%a3/"
url = " https://artyt.me/tag/%e0%b8%a5%e0%b9%89%e0%b8%b2%e0%b8%87%e0%b8%9f%e0%b8%b4%e0%b8%a5%e0%b9%8c%e0%b8%a1/"
# url = "https://pantip.com/topic/41822838"

webReq = requests.get(url)

In [77]:
# def scrape_raw_text(webReq):
#     """Input responsed requested object (webReq = requests.get(url)), Return as raw text string"""
#     soup_obj = BeautifulSoup(webReq.text, "html.parser")
#     article = ''
#     # Get Header Text
#     for raw in soup_obj.find_all('h1'):
#         article = article + " " + raw.text
#     # Get Article text
#     for raw in soup_obj.find_all('p'):
#         article = article + " " + raw.text
#     return article

def scrape_raw_text(webReq):
    """Input responsed requested object (webReq = requests.get(url)), Return as raw text string"""
    soup_obj = BeautifulSoup(webReq.text, "html.parser")
    article = ''
    # Get Header Text and Article text
    for raw in soup_obj.find_all(['h1', 'p']):
        article += " " + raw.text
    return article


# Compilation class for scrape url

In [75]:
from tqdm import tqdm
import concurrent.futures
import requests
from bs4 import BeautifulSoup
import mysql.connector
import time
import validators

# url = 'https://www.35mmc.com/20/05/2016/halina-af700-review/'
# url = "https://fstoppers.com/film/how-was-i-missing-medium-format-point-and-shoot-short-term-review-fujifilm-ga645-470116"
# url = "https://www.trustedreviews.com/reviews/nikon-z9"
# url = "https://artyt.me/2022/03/21/%e0%b8%a3%e0%b8%b5%e0%b8%a7%e0%b8%b4%e0%b8%a7-10-%e0%b8%81%e0%b8%a5%e0%b9%89%e0%b8%ad%e0%b8%87%e0%b8%9f%e0%b8%b4%e0%b8%a5%e0%b9%8c%e0%b8%a1%e0%b8%97%e0%b8%b5%e0%b9%88%e0%b8%84%e0%b8%a7%e0%b8%a3/"
url = " https://artyt.me/tag/%e0%b8%a5%e0%b9%89%e0%b8%b2%e0%b8%87%e0%b8%9f%e0%b8%b4%e0%b8%a5%e0%b9%8c%e0%b8%a1/"
# url = "https://pantip.com/topic/41822838"

webReq = requests.get(url)
    
def scrape_raw_text(webReq):
    """Input responsed requested object (webReq = requests.get(url)), Return as raw text string"""
    soup_obj = BeautifulSoup(webReq.text, "html.parser")
    article = ''
    # Get Header Text and Article text
    for raw in soup_obj.find_all(['h1', 'p']):
        article += " " + raw.text
    return article

def scrape_all_link(webReq):
    """Input responsed requested object (webReq = requests.get(url)), Return as list of all URLs string"""
    soup_obj = BeautifulSoup(webReq.text, "html.parser")
    return_url = []
    for url_obj in soup_obj.find_all('a'):
        url = url_obj.get('href')
        if url == None:
            pass
        elif (url.find("https:")==0 or url.find("http:")==0) and (url not in return_url) :
            return_url.append(url)   
    return return_url


In [80]:
from bs4 import BeautifulSoup
import requests
import validators

class WebScraper:
    def __init__(self, webReq):
        self.soup_obj = BeautifulSoup(webReq.text, "html.parser")
        
    def scrape_raw_text(self):
        """Return raw text string"""
        article = ''
        for raw in self.soup_obj.find_all(['h1', 'p']):
            article += " " + raw.text
        return article
    
    def scrape_all_link(self):
        """Return list of all URLs string"""
        return_url = []
        for url_obj in self.soup_obj.find_all('a'):
            url = url_obj.get('href')
            if url == None:
                pass
            elif (url.find("https:")==0 or url.find("http:")==0) and (url not in return_url) :
                return_url.append(url)   
        return return_url

In [83]:
url = "https://artyt.me/2022/03/21/%e0%b8%a3%e0%b8%b5%e0%b8%a7%e0%b8%b4%e0%b8%a7-10-%e0%b8%81%e0%b8%a5%e0%b9%89%e0%b8%ad%e0%b8%87%e0%b8%9f%e0%b8%b4%e0%b8%a5%e0%b9%8c%e0%b8%a1%e0%b8%97%e0%b8%b5%e0%b9%88%e0%b8%84%e0%b8%a7%e0%b8%a3/"

webReq = requests.get(url)
scraper = WebScraper(webReq)
article = scraper.scrape_raw_text()
links = scraper.scrape_all_link()

# Optimized Class For scrape

In [196]:
import validators
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse

class pageScraper:
    """Class for Scrape single page, Return URL, all backlinks and Raw Text"""
    def __init__(self):
        # Set allowed domain
        self.allowed_domain = [
            "artyt.me",
            "www.35mmc.com",
            "www.dpreview.com"
        ]
    
    def get_raw_html(self, url):
        """get raw html soup obj"""
        # webReq = requests.get(url)
        return BeautifulSoup((requests.get(url)).text, "html.parser")
    
    def scrape_raw_text(self, soup_obj):
        """Return raw text string from bs4 boject"""
        return ' '.join([raw.text for raw in soup_obj.find_all(['h1', 'p'])])
    
    def scrape_all_urls(self, soup_obj):
        """Return list of all URLs string from bs4 object"""
        return_url = []
        for url_obj in soup_obj.find_all('a'):
            url = url_obj.get('href')
            if (url is not None) and validators.url(url) and (url not in return_url) and ((urlparse(url).netloc) in self.allowed_domain):
                return_url.append(url)
                # print(url)
                # print(urlparse(url).netloc)
        return return_url
    
    def scrape_page(self, url):
        """Return a dictionary of url and raw text"""
        raw_soup_html = self.get_raw_html(url)
        return {
            "url" : url,
            "backlinks" : self.scrape_all_urls(raw_soup_html),
            "rawText" : self.scrape_raw_text(raw_soup_html)
        }
        

In [199]:
ps = pageScraper()
ps.scrape_page("https://www.example.com/")
# ps.scrape_page("https://www.example.com/")["url"]

{'url': 'https://www.example.com/',
 'backlinks': [],
 'rawText': 'Example Domain This domain is for use in illustrative examples in documents. You may use this\n    domain in literature without prior coordination or asking for permission. More information...'}